## Question 1 : 
I did the calculus on paper here are my results :

$q_1(s_1,a_1) = 10.6 , q_1(s_1,a_2) = 11.2$ \
$q_1(s_2,a_1) = 4.3 , q_1(s_2,a_2) = 4.3$ \
We have $\pi^1(s_1)=a_1$ and $\pi^1(s_2)=a_1 or a_2$

Using the same method, $\pi^2(s_1)=a_1$ and $\pi^2(s_2)=a_2$

To evaluate the deterministic policy for $k>2$ we evaluate $q(s_1,a_1) - q(s_1,a_2)$ and $q(s_2,a_1) - q(s_2,a_2)$

The key point is that with value iteration, at each iteration the value function is bigger than at the precedent step, for each state.

$$q(s_2,a_1) - q(s_2,a_2) = 2 - 0.2V(s_1)$$ and as we have $V(s_1)>10, \pi(s_2) = a_2$

Same, $$q(s_1,a_1) - q(s_1,a_2) = 0.1V(s_1) + 0.4V(s_2) - 2$$ as we have $V(s_1) > 10$ and $ V(s_2)>4$, $\pi(s_1)=a_1$

## Question 4

In [1]:
import os
os.chdir('/Users/Alex/Desktop/Documents_4A/Winter_quarter_1/MS&E_346/RL_book/')

In [2]:
from dataclasses import dataclass
from typing import Tuple, Dict, Mapping
from rl.markov_decision_process import FiniteMarkovDecisionProcess
from rl.policy import FiniteDeterministicPolicy
from rl.markov_process import FiniteMarkovProcess, FiniteMarkovRewardProcess
from rl.distribution import Categorical
from scipy.stats import poisson

In [15]:
## Defining the markov decision process

k1 = 2
k2 = 5

@dataclass(frozen=True)
class DoubleInventoryState:
    on_hand_1: int
    on_hand_2: int
    on_order_1: int
    on_order_2:int

    def inventory_position(self) -> int:
        return (self.on_hand_1 + self.on_order_1, self.on_hand_2 + self.on_order_2)


InvOrderMapping = Mapping[
    DoubleInventoryState,
    Mapping[int, Categorical[Tuple[DoubleInventoryState, float]]]
]


class DoubleInventoryMDPCap(FiniteMarkovDecisionProcess[DoubleInventoryState, int]):

    def __init__(
        self,
        capacity_1: int,
        capacity_2: int,
        poisson_lambda_1: float,
        poisson_lambda_2:float,
        holding_cost_1: float,
        holding_cost_2: float,
        stockout_cost_1: float,
        stockout_cost_2: float
    ):
        self.capacity_1: int = capacity_1
        self.poisson_lambda_1: float = poisson_lambda_1
        self.holding_cost_1: float = holding_cost_1
        self.stockout_cost_1: float = stockout_cost_1
        self.capacity_2: int = capacity_2
        self.poisson_lambda_2: float = poisson_lambda_2
        self.holding_cost_2: float = holding_cost_2
        self.stockout_cost_2: float = stockout_cost_2

        self.poisson_distr_1 = poisson(poisson_lambda_1)
        self.poisson_distr_2 = poisson(poisson_lambda_2)
        super().__init__(self.get_action_transition_reward_map())

    def get_action_transition_reward_map(self) -> InvOrderMapping:
        d: Dict[DoubleInventoryState, Dict[Tuple[int,int,int], Categorical[Tuple[DoubleInventoryState,
                                                            float]]]] = {}
        for alpha_1 in range(self.capacity_1 + 1):
            for beta_1 in range(self.capacity_1 + 1 - alpha_1):
                for alpha_2 in range(self.capacity_2 + 1):
                    for beta_2 in range(self.capacity_2 + 1 - alpha_2):
                        state: DoubleInventoryState = DoubleInventoryState(alpha_1, beta_1,alpha_2,beta_2)
                        ip: tuple = state.inventory_position()
                        base_reward_1: float = - self.holding_cost_1 * alpha_1
                        base_reward_2: float = - self.holding_cost_2 * alpha_2
                        d1: Dict[Tuple[int,int,int], Categorical[Tuple[DoubleInventoryState, float]]] = {}
                            
                        for order_1 in range(self.capacity_1 - ip[0] + 1): #Orders of 1
                            for order_2 in range(self.capacity_1 - ip[1] + 1): # Orders of 2
                                for transfer in range(-ip[1],ip[0]): #Convention : transfer is positive from store 1 to 2
                                    sr_probs_dict: Dict[Tuple[DoubleInventoryState, float], float] = {}
                                    for i in range(ip[0]):
                                        for j in range(ip[1]):
                                            sr_probs_dict[(DoubleInventoryState(ip[0] - i, ip[1] - j, order_1,order_2), base_reward_1 + base_reward_2 - k1*(transfer!=0) - k2*(order_1!=0) - k2*(order_2!=0))] =\
                                            self.poisson_distr_1.pmf(i)*self.poisson_distr_2.pmf(j)
                                    for j in range(ip[1]):
                                        prob_1: float = 1 - self.poisson_distr_1.cdf(ip[0] - 1)
                                        reward_1: float = base_reward_1 - self.stockout_cost_1 *(prob_1 * (self.poisson_lambda_1 - ip[0]) + ip[0] * self.poisson_distr_1.pmf(ip[0]))
                                        sr_probs_dict[(DoubleInventoryState(0, ip[1] - j, order_1,order_2), reward_1 + base_reward_2 - k1*(transfer!=0) - k2*(order_1!=0) - k2*(order_2!=0))] =\
                                            prob_1*self.poisson_distr_2.pmf(j)
                                    for i in range(ip[0]):
                                        prob_2: float = 1 - self.poisson_distr_1.cdf(ip[1] - 1)
                                        reward_2: float = base_reward_2 - self.stockout_cost_2 *(prob_2 * (self.poisson_lambda_2 - ip[1]) + ip[1] * self.poisson_distr_2.pmf(ip[1]))
                                        sr_probs_dict[(DoubleInventoryState(ip[0]-i, 0, order_1,order_2), base_reward_1 + reward_2 - k1*(transfer!=0) - k2*(order_1!=0) - k2*(order_2!=0))] =\
                                            prob_2*self.poisson_distr_1.pmf(i)
                                    prob_1: float = 1 - self.poisson_distr_1.cdf(ip[0] - 1)
                                    prob_2: float = 1 - self.poisson_distr_1.cdf(ip[1] - 1)
                                    reward_1: float = base_reward_1 - self.stockout_cost_1 *(prob_1 * (self.poisson_lambda_1 - ip[0]) + ip[0] * self.poisson_distr_1.pmf(ip[0]))
                                    reward_2: float = base_reward_2 - self.stockout_cost_2 *(prob_2 * (self.poisson_lambda_2 - ip[1]) + ip[1] * self.poisson_distr_2.pmf(ip[1]))
                                    sr_probs_dict[(DoubleInventoryState(0, 0, order_1,order_2), reward_1 + reward_2 - k1*(transfer!=0) - k2*(order_1!=0) - k2*(order_2!=0))] =\
                                        prob_1*prob_2
                                    d1[order_1,order_2,transfer] = Categorical(sr_probs_dict)

                        d[state] = d1
        return d


In [36]:
#Parameters
user_capacity_1 = 3
user_poisson_lambda_1 = 1.0
user_holding_cost_1 = 1.0
user_stockout_cost_1 = 10.0

user_capacity_2 = 2
user_poisson_lambda_2 = 1.5
user_holding_cost_2 = 2.0
user_stockout_cost_2 = 8.0

di_mdp = DoubleInventoryMDPCap(capacity_1=user_capacity_1,
        poisson_lambda_1=user_poisson_lambda_1,
        holding_cost_1=user_holding_cost_1,
        stockout_cost_1=user_stockout_cost_1,
        capacity_2=user_capacity_2,
        poisson_lambda_2=user_poisson_lambda_2,
        holding_cost_2=user_holding_cost_2,
        stockout_cost_2=user_stockout_cost_2)

In [32]:
print(di_mdp)

From State DoubleInventoryState(on_hand_1=0, on_hand_2=0, on_order_1=0, on_order_2=0):
From State DoubleInventoryState(on_hand_1=0, on_hand_2=0, on_order_1=0, on_order_2=1):
  With Action (0, 0, -1):
    To [State DoubleInventoryState(on_hand_1=0, on_hand_2=1, on_order_1=0, on_order_2=0) and Reward -12.000] with Probability 0.261
    To [State DoubleInventoryState(on_hand_1=0, on_hand_2=0, on_order_1=0, on_order_2=0) and Reward -17.206] with Probability 0.739
  With Action (0, 1, -1):
    To [State DoubleInventoryState(on_hand_1=0, on_hand_2=1, on_order_1=0, on_order_2=1) and Reward -17.000] with Probability 0.261
    To [State DoubleInventoryState(on_hand_1=0, on_hand_2=0, on_order_1=0, on_order_2=1) and Reward -22.206] with Probability 0.739
  With Action (1, 0, -1):
    To [State DoubleInventoryState(on_hand_1=0, on_hand_2=1, on_order_1=1, on_order_2=0) and Reward -17.000] with Probability 0.261
    To [State DoubleInventoryState(on_hand_1=0, on_hand_2=0, on_order_1=1, on_order_2=0)

In [33]:
#Optimal Value functions
from rl.dynamic_programming import evaluate_mrp_result
from rl.dynamic_programming import policy_iteration_result
from rl.dynamic_programming import value_iteration_result

In [34]:
value_iteration_result(di_mdp, gamma=1)

ValueError: converged called on an empty iterator